# Clean and merge data

In [1]:
# First party libraries
import time
import csv
import os

# 3rd Party Libraries
import pandas as pd
import numpy as np

In [2]:
csv_files = ['data/Steamboat_15K_2016_results.csv',\
             'data/Steamboat_15K_2018_results.csv',\
             'data/Steamboat_15K_2022_results.csv',\
             'data/Steamboat_15K_2017_results.csv',\
             'data/Steamboat_15K_2019_results.csv',\
             'data/Steamboat_15K_2023_results.csv'
            ]


In [3]:
f_name = csv_files[0]
df = pd.read_csv(f_name)

In [4]:
df.head()

,name,bib,gender,age,time
0,Zach Plank,9755,M,21.0,48:18
1,Dan O'Keefe,9753,M,20.0,49:28
2,Omar Gomez,9757,M,21.0,50:46
3,Nolan McKenna,9754,M,20.0,51:46
4,Dan Regalado,9587,M,29.0,52:02


In [5]:
race_yr = f_name.split('_')[2]
print(race_yr)

2016


In [6]:
df['year'] = race_yr
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 822 entries, 0 to 821
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   name    822 non-null    object 
 1   bib     822 non-null    int64  
 2   gender  822 non-null    object 
 3   age     821 non-null    float64
 4   time    822 non-null    object 
 5   year    822 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 38.7+ KB
None


,name,bib,gender,age,time,year
0,Zach Plank,9755,M,21.0,48:18,2016
1,Dan O'Keefe,9753,M,20.0,49:28,2016
2,Omar Gomez,9757,M,21.0,50:46,2016
3,Nolan McKenna,9754,M,20.0,51:46,2016
4,Dan Regalado,9587,M,29.0,52:02,2016


In [7]:
tm = '48:18'

In [8]:
tm_split = tm.split(':')[::-1]
tm_seconds = tm_split[0]
if len(tm_split) >1:
    tm_minutes = tm_split[1]
    if len(tm_split) >2:
        tm_hours = tm_split[2]
    else:
        tm_hours = 0
else:
    tm_minutes = 0
    tm_hour = 0
print(f'Time {tm_hours} hours, {tm_minutes} minutes, {tm_seconds} seconds')
tot_time_seconds = int(tm_hours) * 3600 + int(tm_minutes) * 60 + int(tm_seconds)
print(f'Total time in seconds: {tot_time_seconds}')

Time 0 hours, 48 minutes, 18 seconds
Total time in seconds: 2898


In [9]:
tm_to_seconds_conv = [1, 60, 3600] # list of conversion amounts for seconds, minutes, hours to seconds
tm_split = tm.split(':')[::-1] # reverse the list so seconds are first
tot_time_seconds = 0
for idx, tm_entry in enumerate(tm_split):
    tot_time_seconds = tot_time_seconds + int(tm_entry) * tm_to_seconds_conv[idx]
print(f'Total time in seconds: {tot_time_seconds}')

Total time in seconds: 2898


In [10]:
tm_to_seconds_conv = [1, 60, 3600] # list of conversion amounts for seconds, minutes, hours to seconds
tot_time_seconds = 0
for idx, tm_entry in enumerate(tm.split(':')[::-1]):
    tot_time_seconds = tot_time_seconds + int(tm_entry) * tm_to_seconds_conv[idx]
print(f'Total time in seconds: {tot_time_seconds}')

Total time in seconds: 2898


## Get time in seconds

In [11]:
df[['tm_split_s','tm_split_m','tm_split_h']] = df['time'].apply(lambda x:pd.Series(x.split(':')[::-1]))
df[['tm_split_s','tm_split_m','tm_split_h']] = \
    df[['tm_split_s','tm_split_m','tm_split_h']].fillna(0).astype('int64')
df['time_seconds'] = df['tm_split_s'] + df['tm_split_m']  *60 + df['tm_split_h'] *3600
df.head()

,name,bib,gender,age,time,year,tm_split_s,tm_split_m,tm_split_h,time_seconds
0,Zach Plank,9755,M,21.0,48:18,2016,18,48,0,2898
1,Dan O'Keefe,9753,M,20.0,49:28,2016,28,49,0,2968
2,Omar Gomez,9757,M,21.0,50:46,2016,46,50,0,3046
3,Nolan McKenna,9754,M,20.0,51:46,2016,46,51,0,3106
4,Dan Regalado,9587,M,29.0,52:02,2016,2,52,0,3122


## Convert field types and remove invalid records

### Check for any ages that are not populated and remove those entries

In [12]:
df[df['age'].isna()].head()

,name,bib,gender,age,time,year,tm_split_s,tm_split_m,tm_split_h,time_seconds
551,Samantha Richrath,3132,F,NaN,1:33:02,2016,2,33,1,5582


In [13]:
df.dropna(subset=['age'], inplace=True)

In [14]:
convert_dict = {'bib':'object','age':'int64'}
df = df.astype(convert_dict)
df.head()

,name,bib,gender,age,time,year,tm_split_s,tm_split_m,tm_split_h,time_seconds
0,Zach Plank,9755,M,21,48:18,2016,18,48,0,2898
1,Dan O'Keefe,9753,M,20,49:28,2016,28,49,0,2968
2,Omar Gomez,9757,M,21,50:46,2016,46,50,0,3046
3,Nolan McKenna,9754,M,20,51:46,2016,46,51,0,3106
4,Dan Regalado,9587,M,29,52:02,2016,2,52,0,3122


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 821 entries, 0 to 821
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          821 non-null    object
 1   bib           821 non-null    object
 2   gender        821 non-null    object
 3   age           821 non-null    int64 
 4   time          821 non-null    object
 5   year          821 non-null    object
 6   tm_split_s    821 non-null    int64 
 7   tm_split_m    821 non-null    int64 
 8   tm_split_h    821 non-null    int64 
 9   time_seconds  821 non-null    int64 
dtypes: int64(5), object(5)
memory usage: 70.6+ KB


# Drop unneeded fields

In [16]:
df.drop(columns=['name','bib', 'tm_split_s', 'tm_split_m', 'tm_split_h'], inplace=True)

# Cleaned data

In [17]:
df.head()

,gender,age,time,year,time_seconds
0,M,21,48:18,2016,2898
1,M,20,49:28,2016,2968
2,M,21,50:46,2016,3046
3,M,20,51:46,2016,3106
4,M,29,52:02,2016,3122


In [18]:
df.tail()

,gender,age,time,year,time_seconds
817,F,43,2:24:45,2016,8685
818,F,30,2:26:16,2016,8776
819,F,71,2:28:13,2016,8893
820,M,35,2:30:12,2016,9012
821,M,74,2:32:51,2016,9171
